In [1]:
import numpy as np
import pandas as pd

import cv2

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold

In [2]:
data_path = '../open/'
info = pd.read_excel(data_path+'clinical_info.xlsx')

df_train = pd.read_csv(data_path+'train.csv')
df_test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')

In [3]:
info

,항목,코딩,설명
0,ID,NaN,부여번호
1,나이,NaN,진단시 나이
2,수술연월일,연-월-일,NaN
3,진단명,1: ductal\n2: lobular\n3. mucinous \n4. other,histologic type
4,암의 위치,1: Right\n2: Left\n3: Both,NaN
5,암의 개수,1: single\n2: multiple,NaN
6,암의 장경,NaN,"암 Size (mm), 여러 개의 암일 경우 가장 큰 종양의 장경"
7,NG,1: NG1\n2: NG2\n3: NG3,"1은 핵의 모양이 정상에 가까운 경우, 2는 중간, 3은 정상으로부터 매우 달라진경..."
8,HG,1: HG1\n2: HG2\n3: HG3\n4. not graded,"Grade 1 (scores of 3, 4, or 5): 조직학적 분화도가 좋아서 ..."
9,HG_score_1,1: score 1\n2: score 2\n3: score 3\n4: not graded,(Tubule formation)\nScore 1 (>75% of tumor are...


In [4]:
df_train = pd.read_csv(data_path+'train.csv')
df_test = pd.read_csv(data_path+'test.csv')

# preprocess outlier
df_train['PR_Allred_score'] = df_train['PR_Allred_score'].where((0<=df_train['PR_Allred_score']) & (df_train['PR_Allred_score']<=8))

for col in ['NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_type', 'ER_Allred_score', 'PR_Allred_score', 'HER2_SISH_ratio']:
    df_train[col].fillna(0, inplace=True)
    df_test[col].fillna(0, inplace=True)

df_test['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)
df_train['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)

df_train['BRCA_mutation'] = df_train['BRCA_mutation'].fillna(1)
df_test['BRCA_mutation'] = df_test['BRCA_mutation'].fillna(1)

for col in ['T_category', 'HER2', 'HER2_IHC', 'HER2_SISH', 'KI-67_LI_percent']:
    df_train[col].fillna(-1, inplace=True)
    df_test[col].fillna(-1, inplace=True)

    df_train[col]+=1
    df_test[col]+=1

df_train[['ER', 'PR']] = df_train[['ER', 'PR']].fillna(0)
# remove ER, PR nan value
# df_train = df_train.drop(266).reset_index(drop=True)

In [1]:
2e-3

0.002

In [2]:
import torch
import torch.nn as nn

In [18]:
a = torch.rand([4, 2])

In [25]:
a

tensor([[0.9535, 0.8353],
        [0.6862, 0.1089],
        [0.7766, 0.9861],
        [0.3188, 0.0976]])

In [27]:
torch.softmax(a, dim=1)

tensor([[0.5295, 0.4705],
        [0.6404, 0.3596],
        [0.4478, 0.5522],
        [0.5551, 0.4449]])

In [393]:
pd.options.display.max_columns=100

In [394]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [396]:
df_test.isnull().sum()[lambda x: x>0]

Series([], dtype: int64)

In [417]:
df_train = pd.read_csv(data_path+'train.csv')
df_test = pd.read_csv(data_path+'test.csv')

# preprocess outlier
df_train['PR_Allred_score'] = df_train['PR_Allred_score'].where((0<=df_train['PR_Allred_score']) & (df_train['PR_Allred_score']<=8))

for col in ['NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_type', 'ER_Allred_score', 'PR_Allred_score', 'HER2_SISH_ratio']:
    df_train[col].fillna(0, inplace=True)
    df_test[col].fillna(0, inplace=True)

df_test['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)
df_train['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)

df_train['BRCA_mutation'] = df_train['BRCA_mutation'].fillna(1)
df_test['BRCA_mutation'] = df_test['BRCA_mutation'].fillna(1)

for col in ['T_category', 'HER2', 'HER2_IHC', 'HER2_SISH', 'KI-67_LI_percent']:
    df_train[col].fillna(-1, inplace=True)
    df_test[col].fillna(-1, inplace=True)

    df_train[col]+=1
    df_test[col]+=1

# remove ER, PR nan value
df_train = df_train.drop(266).reset_index(drop=True)


FOLDS = 10

def basic_set():
    use_features = [
                    '나이', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
                    'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 
                    'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 
                    'KI-67_LI_percent', 
                    'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 
                    # 'BRCA_mutation2',
                    ]

    cat_features = ['나이', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1',
                            'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type',
                            'T_category', 'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score',
                            'HER2', 'HER2_IHC', 'HER2_SISH','BRCA_mutation']
    cat_features = ['진단명']
    return use_features, cat_features

df_train['due_date'] = 2022 - pd.to_datetime(df_train['수술연월일']).dt.year
df_test['due_date'] = 2022 - pd.to_datetime(df_test['수술연월일']).dt.year
# df_train['due_date2'] = (2022-pd.to_datetime(df_train['수술연월일']).dt.year)*12 + pd.to_datetime(df_train['수술연월일']).dt.month
# df_test['due_date2'] = (2022-pd.to_datetime(df_test['수술연월일']).dt.year)*12 + pd.to_datetime(df_test['수술연월일']).dt.month

df_train['date_year'] = pd.to_datetime(df_train['수술연월일']).dt.year
df_test['date_year'] = pd.to_datetime(df_test['수술연월일']).dt.year

# df_train['암의 장경/개수'] = df_train['암의 장경'] / df_train['암의 개수']
# df_test['암의 장경/개수'] = df_test['암의 장경'] / df_test['암의 개수']

skf = StratifiedKFold(n_splits=FOLDS, random_state=42, shuffle=True)
splits = list(skf.split(df_train, df_train['N_category']))
# labels = df_train['N_category'].astype(str) + '_' + df_train['due_date'].astype(str)
# splits = list(skf.split(df_train, labels))


fi_df = pd.DataFrame()
oof = np.zeros(len(df_train))
preds = np.zeros(len(df_test))

for fold in range(FOLDS):
    print('start fold :', fold)
    test = df_test.copy()
    
    use_features, cat_features = basic_set()
    use_features += ['due_date', 'date_year']
    # use_features += ['due_date', 'date_year', '암의 장경/개수']

    tt = df_train.loc[splits[fold][0], use_features].reset_index(drop=True)
    tt_target = df_train.loc[splits[fold][0], 'N_category'].values
    vv = df_train.loc[splits[fold][1], use_features].reset_index(drop=True)
    vv_target = df_train.loc[splits[fold][1], 'N_category'].values

    for df in [tt, vv, test]:
        df['나이'] = pd.cut(df['나이'], [20, 30, 40, 50, 60, 70, 80, 100])
        # df['암의 장경'] = pd.cut(df['암의 장경'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])
        df['암의 장경'] = pd.cut(df['암의 장경'], [0, 20, 40, 60, 80, 100, 200])
        # df['암의 장경/개수'] = pd.cut(df['암의 장경/개수'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])
        # df['KI-67_LI_percent'] = pd.cut(df['KI-67_LI_percent'], [0, 20, 40, 60, 80, 100])
        
    # for col in ['나이', '암의 장경', '암의 장경/개수']:
    # for col in ['나이', '암의 장경', 'KI-67_LI_percent']:
    for col in ['나이', '암의 장경']:
        tmp_dict = {j:i for i,j in enumerate(np.unique(tt[col].astype(str)))}
        tt[col] = tt[col].astype(str).map(tmp_dict)
        vv[col] = vv[col].astype(str).map(tmp_dict)
        test[col] = test[col].astype(str).map(tmp_dict)
    
    # tt['암의 장경2'] = tt['암의 장경'].isnull()
    # vv['암의 장경2'] = vv['암의 장경'].isnull()
    # test['암의 장경2'] = test['암의 장경'].isnull()

    # for df in [tt, vv, test]:
    #     # df['나이_due_date'] = df['나이'].astype(str) + '_' + df['due_date'].astype(str)
    #     df['암'] = df['암의 위치'].astype(str) + '_' + df['암의 장경'].astype(str)
    # df = pd.concat([tt, vv])
    # tmp_dict = {v:i for i, v in enumerate(np.unique(df['암']))}
    # for col in ['암']:
    #     tt[col] = tt[col].astype(str).map(tmp_dict)
    #     vv[col] = vv[col].astype(str).map(tmp_dict)
    #     test[col] = test[col].astype(str).map(tmp_dict)
    
    # use_features += ['암']
    
    # tmp_dict = df_train.loc[splits[fold][0]].groupby('due_date')['N_category'].mean()
    # tt['진단명_target'] = tt['due_date'].map(tmp_dict)
    # vv['진단명_target'] = vv['due_date'].map(tmp_dict)
    # tmp_dict = df_train.groupby('due_date')['N_category'].mean()
    # test['진단명_target'] = test['due_date'].map(tmp_dict)
    # use_features += ['진단명_target']

    test = test[use_features].reset_index(drop=True)

    # lgb = LGBMClassifier(
    #                     n_estimators=5000,
    #                     learning_rate=0.03,
    #                     max_depth=-1,
    #                     num_leaves=64,
    #                     )

    # lgb.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=100)
    # oof[splits[fold][1]] += lgb.predict_proba(vv)[:, 1]
    # preds += lgb.predict_proba(test)[:, 1] / 5
    # fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, lgb.feature_importances_), columns=['feature', 'value'])])
    # tt[cat_features] = tt[cat_features].fillna(-99).astype(int)
    # vv[cat_features] = vv[cat_features].fillna(-99).astype(int)
    cat = CatBoostClassifier(
                            n_estimators=5000,
                            learning_rate=0.03,
                            # objective='logloss',
                            # eval_metric='F1',
                            # reg_lambda=0.01,
                            # random_seed=42,
                            allow_writing_files=False,
                            logging_level='Silent',
                            )
    cat.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=500,
            # cat_features=cat_features
            
            )
    oof[splits[fold][1]] += cat.predict_proba(vv)[:, 1]
    preds += cat.predict_proba(test)[:, 1] / FOLDS
    fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, cat.feature_importances_), columns=['feature', 'value'])])
    
    # use_features.remove('암')

start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4
start fold : 5
start fold : 6
start fold : 7
start fold : 8
start fold : 9


In [423]:
from sklearn.metrics import f1_score
f1_score(df_train['N_category'],
        np.where(oof>0.55, 1, 0),
        average='weighted'
        )

0.8396934282424039

In [425]:
from sklearn.metrics import f1_score
f1_score(df_train['N_category'],
        np.where(oof>0.55, 1, 0),
        average='weighted'
        )

0.8396934282424039

In [434]:
from sklearn.metrics import f1_score
f1_score(df_train['N_category'],
        np.where(oof>0.4, 1, 0),
        average='weighted'
        )

0.8176384902379277

In [414]:
fi_df.groupby('feature').mean().reset_index().sort_values('value', ascending=False)

,feature,value
19,due_date,16.677493
18,date_year,13.121424
14,NG,11.432816
11,HG_score_2,6.966082
13,KI-67_LI_percent,5.587786
22,암의 위치,3.982920
16,PR_Allred_score,3.761313
23,암의 장경,3.517789
1,DCIS_or_LCIS_type,3.505500
20,나이,3.345240


In [428]:
pd.Series(np.where(oof>0.55, 1, 0)).value_counts(True)

0    0.533534
1    0.466466
dtype: float64

In [431]:
df_train['N_category'].value_counts(True)

1    0.514515
0    0.485485
Name: N_category, dtype: float64

In [433]:
pd.Series(np.where(preds>0.4, 1, 0)).value_counts(True)

1    0.512
0    0.488
dtype: float64

In [435]:
pd.Series(np.where(preds>0.4, 1, 0)).value_counts(True)

1    0.512
0    0.488
dtype: float64

In [436]:
sub['N_category'] = np.where(preds>=0.4, 1, 0)

In [437]:
sub.to_csv('../submit/catboost_data_cleaning_10folds__pp4.csv', index=False)

In [124]:
np.save('../submit/cat_oof.npy', oof)

In [125]:
np.save('../submit/cat_preds.npy', preds)

In [ ]:
df_train['due_date'] = 2022 - pd.to_datetime(df_train['수술연월일']).dt.year

In [ ]:
df_train.groupby('N_category')['암의 개수'].value_counts()

In [50]:
df_train.groupby('N_category')['KI-67_LI_percent'].value_counts().unstack()

KI-67_LI_percent,0.00,0.15,0.19,0.31,0.40,0.42,0.55,0.90,0.91,1.00,1.07,1.11,1.19,1.27,1.42,1.49,1.58,1.60,1.76,1.84,1.98,2.00,2.06,2.08,2.14,2.30,2.32,2.38,2.60,2.66,3.00,3.03,3.08,3.25,3.32,3.40,3.45,3.50,3.51,3.53,3.60,3.69,3.81,3.85,3.98,4.00,4.23,4.24,4.25,4.27,...,44.08,44.27,44.67,44.76,44.92,45.00,45.27,45.37,45.55,45.64,46.99,47.00,47.79,47.92,48.66,49.98,50.00,50.82,50.96,51.78,52.00,52.87,53.04,54.55,54.83,56.99,57.40,57.65,60.00,60.12,60.60,61.69,62.04,64.90,66.00,66.26,70.00,75.00,77.22,79.27,79.66,80.00,80.65,81.30,85.71,87.37,89.17,90.00,95.00,97.00
N_category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,34.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,1.0,14.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,20.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,4.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,1.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN
1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,22.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,11.0,1.0,NaN,1.0,1.0,NaN,NaN,2.0,1.0,7.0,NaN,1.0,NaN,NaN,NaN,2.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,2.0,NaN,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,3.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,4.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,6.0,1.0,1.0,1.0,NaN,3.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0


In [ ]:
df_train.groupby('N_category')['DCIS_or_LCIS_여부'].value_counts()

In [ ]:
df_train.groupby('N_category')['due_date'].value_counts()

In [ ]:
img = cv2.imread('../open/train_imgs/BC_01_0015.png')
mask = cv2.imread('../open/train_masks/BC_01_0015.png')
plt.subplots(2,1)
plt.subplot(2,1,1)
plt.imshow(img)
plt.subplot(2,1,2)
plt.imshow(mask)
plt.show()

In [ ]:
import albumentations as A

In [ ]:
trans = A.VerticalFlip(p=1.0)

trans = A.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0)

In [ ]:
trans_img = trans(image=img)['image']

In [ ]:
plt.imshow(trans_img)

In [ ]:
plt.imshow(trans_img)

In [ ]:
df_train[lambda x: x['mask_path']!='-'].head()

In [ ]:
img = cv2.imread('../open/test_imgs/BC_01_0011.png')
plt.imshow(img)
plt.show()

In [ ]:
df_test

In [100]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

In [103]:
a = np.random.rand(2, 2)
a = np.array([[1,2],[3,4]])
a

array([[1, 2],
       [3, 4]])

In [94]:
param1 = nn.Parameter(torch.rand(2, 1))
param2 = nn.Parameter(torch.rand(2, 1))
param1

Parameter containing:
tensor([[0.6711],
        [0.1316]], requires_grad=True)

In [48]:
b = torch.tensor(a)[:,:,None] * param1
b = b + param2

In [104]:
torch.tensor(a)[:,:,None]

tensor([[[1],
         [2]],

        [[3],
         [4]]])

In [108]:
d = torch.tensor(a)[:,:,None] * param1

In [106]:
param2

Parameter containing:
tensor([[0.2899],
        [0.3521]], requires_grad=True)

In [107]:
torch.tensor(a)[:,:,None] * param1 + param2

tensor([[[0.9610],
         [0.6153]],

        [[2.3032],
         [0.8785]]], grad_fn=<AddBackward0>)

In [44]:
b.shape

torch.Size([10, 5, 32])

In [37]:
b.shape

torch.Size([10, 5, 32])

In [35]:
a.shape

(10, 5)